## FAISS

https://github.com/facebookresearch/faiss/wiki

#### Installation

In [1]:
# !pip3 install faiss-cpu
# !pip3 install faiss-gpu

### The Data-base is chosen large for experimentes

In [1]:
import numpy as np
d = 64                           # dimension
nb = int(25734008/2)                    # database size
nq = 10000                       # nb of queries
np.random.seed(1234)             # make reproducible
xb = np.random.random((nb, d)).astype('float32')
xb[:, 0] += np.arange(nb) / 1000.
xq = np.random.random((nq, d)).astype('float32')
xq[:, 0] += np.arange(nq) / 1000.

### Running on CPU

In [2]:
import faiss                  # make faiss available
index = faiss.IndexFlatL2(d)   # build the index
index.add(xb)                  # add vectors to the index
print(index.ntotal)

12867004


### Searching 1000 on cpu

In [3]:
%%timeit
k = 1000                          # we want to see 4 nearest neighbors
D, I = index.search(xq[:1], k) # sanity check

796 ms ± 4.74 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


#### Using GPU

In [6]:
res = faiss.StandardGpuResources()  # use a single GPU

In [7]:
gpu_index_flat = faiss.index_cpu_to_gpu(res, 0, index)
gpu_index_flat.add(xb)      

#### GPU can only search upto 2500 entires max

In [8]:
%%timeit
k = 1000                          # we want to see 4 nearest neighbors
D, I = gpu_index_flat.search(xq[:1], k) # sanity check

129 ms ± 25.5 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


### Faster Search

To speed up the search, it is possible to segment the dataset into pieces. We define Voronoi cells in the d-dimensional space, and each database vector falls in one of the cells. At search time, only the database vectors y contained in the cell the query x falls in and a few neighboring ones are compared against the query vector.

This is done via the IndexIVFFlat index. This type of index requires a training stage, that can be performed on any collection of vectors that has the same distribution as the database vectors. In this case we just use the database vectors themselves.

The IndexIVFFlat also requires another index, the quantizer, that assigns vectors to Voronoi cells. Each cell is defined by a centroid, and finding the Voronoi cell a vector falls in consists in finding the nearest neighbor of the vector in the set of centroids. This is the task of the other index, which is typically an IndexFlatL2.

There are two parameters to the search method: nlist, the number of cells, and nprobe, the number of cells (out of nlist) that are visited to perform a search. The search time roughly increases linearly with the number of probes plus some constant due to the quantization.



In [4]:
nlist = 100
k = 4
quantizer = faiss.IndexFlatL2(d)  # the other index
index = faiss.IndexIVFFlat(quantizer, d, nlist)

In [5]:
assert not index.is_trained
index.train(xb)
assert index.is_trained

In [6]:
index.add(xb)                  # add may be a bit slower as well

In [7]:
%%timeit
k = 1000                          # we want to see 4 nearest neighbors
D, I = index.search(xq[:1], k) # sanity check

9.23 ms ± 46.6 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


##### Results can be improves by adding more probe.. current probe=1
which is the correct result. Note that getting a perfect result in this case is merely an artifact of the data distribution, as it is has a strong component on the x-axis which makes it easier to handle. The nprobe parameter is always a way of adjusting the tradeoff between speed and accuracy of the result. Setting nprobe = nlist gives the same result as the brute-force search (but slower).

In [8]:
index.nprobe = 10              # default nprobe is 1, try a few more

In [9]:
%%timeit
k = 1000                          # we want to see 4 nearest neighbors
D, I = index.search(xq[:1], k) # sanity check

113 ms ± 1.58 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)
